# HTBAM Analysis & DB API Tutorial
This notebook walks you through:
1. Loading data via `LocalHtbamDBAPI`
2. Inspecting Data objects
3. Transforming data with `transform_data`
4. Fitting initial rates and dose–response curves
5. Masking poor fits
6. Plotting with `plot_chip`
7. Exporting results to CSV

In [ ]:
# Enable live reloading and inline plots
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from htbam_db_api.htbam_db_api import LocalHtbamDBAPI
from htbam_analysis.analysis.experiment import HTBAMExperiment
from htbam_analysis.analysis.transform import transform_data
from htbam_analysis.analysis.fit import (
    fit_concentration_vs_time,
    fit_luminance_vs_concentration,
    fit_initial_rates_vs_concentration_with_function,
    mm_model,
    inhibition_model
)
from htbam_analysis.analysis.filter import (
    filter_initial_rates_r2_cutoff,
    filter_r2_cutoff
)

## 1. Load & connect to your data
Instantiate `LocalHtbamDBAPI` with your CSVs, then wrap it in an `HTBAMExperiment`.

In [ ]:
root = './examples/ic50_data/'  # adjust as needed
db = LocalHtbamDBAPI(
    standard_curve_data_path = root + 'd3_2_StandardSeries_Analysis.csv',
    standard_name = 'standard_5-FAM',
    standard_substrate = '5-FAM',
    standard_units = 'uM',
    kinetic_data_path = root + 'd3_TitrationSeries_Analysis.csv',
    kinetic_name = 'kinetics_AVI4516',
    kinetic_substrate = 'NSP 4/5',
    kinetic_units = 'nM'
)
exp = HTBAMExperiment(db)
exp.get_run_names()  # list available runs

## 2. Inspect Data objects
Retrieve a run; explore its type, indep_vars, dep_var and dep_var_type.

In [ ]:
raw = exp.get_run('kinetics_AVI4516')
type(raw), raw.indep_vars, raw.dep_var.shape, raw.dep_var_type

## 3. Transform data
Use `transform_data` to compute e.g. product concentration or differences.

In [ ]:
# Example: convert RFU -> concentration using standard curve fits
std_fits = exp.get_run('standard_5-FAM_fits')
kinetic = exp.get_run('kinetics_AVI4516')

# In the transform expression, variables from the first object are prefixed with 'a_' 
# and from the second with 'b_'
prod_conc = transform_data(
    data_objs = [kinetic, std_fits], # (objects a, then b)
    expr = '(a_luminance - b_intercept) / b_slope',
    output_name = 'concentration'
)
exp.set_run('kinetics_conc', prod_conc)
prod_conc.dep_var.shape, prod_conc.dep_var_type

## 4. Fit initial rates & dose–response
Fit linear rates over time, then MM or inhibition models.

In [ ]:
# 4a. fit initial rates
rates3d = fit_concentration_vs_time(prod_conc, start_timepoint=1, end_timepoint=6)
exp.set_run('initial_rates', rates3d)

# 4b. fit IC50
ic50_params, ic50_pred = fit_initial_rates_vs_concentration_with_function(
    data = rates3d,
    model_func = inhibition_model
)
exp.set_run('ic50_params', ic50_params)
exp.set_run('ic50_pred', ic50_pred)

## 5. Mask out low-quality fits
Filter by R² and apply masks.

In [ ]:
mask = filter_r2_cutoff(exp.get_run('ic50_params'), r2_cutoff=0.75)
exp.set_run('ic50_mask', mask)
exp.apply_mask(
    run_name = 'ic50_params',
    dep_variables = ['ic50','r_squared'],
    save_as = 'ic50_params_masked',
    mask_names = ['ic50_mask']
)

## 6. Plotting with `plot_chip`
Visualize standard curves, rates, IC50, and masks.

In [ ]:
# standard curve slopes
exp.plot_standard_curve_chip('standard_5-FAM_fits', 'standard_5-FAM')

# initial rates vs concentration
exp.plot_initial_rates_vs_concentration_chip(
    analysis_name = 'initial_rates',
    model_fit_name = 'ic50_params_masked',
    model_pred_data_name = 'ic50_pred',
    x_log = True
)

## 7. Export results to CSV
Save masked fit parameters and raw data.

In [ ]:
exp.export_run_data_raw('ic50_params_masked', output_dir='.')
exp.export_run_data_processed('ic50_params_masked', output_dir='.')